In [22]:
import pandas as pd
import os



pd.options.display.precision = 4 # show 4 digits precision

In [23]:

# get name of all files in the results directory
file_list = os.listdir('./results')
# Prepare lists to store data
experiments = []
mse_values = []
mae_values = []
infer_time_values = []


for txt_file in file_list:
    
    my_file = './results/' + txt_file
    # Read the file content
    with open(my_file, 'r') as file:
        lines = file.readlines()


    # Parse the content
    for i in range(0, len(lines), 3):
        experiment = lines[i].strip()
        metrics = lines[i+1].strip().split(',')
        


    
        mse = float(metrics[0].split(':')[1])
        mae = float(metrics[1].split(':')[1])
        infer_time = float(metrics[2].split(':')[1])
        

        experiments.append(experiment)
        mse_values.append(mse)
        mae_values.append(mae)
        infer_time_values.append(infer_time)





# Create a DataFrame
df = pd.DataFrame({
    'Experiment_number': experiments,
    'MSE': mse_values,
    'MAE': mae_values,
    'Inference_time':infer_time_values
})


df

,Experiment_number,MSE,MAE,Inference_time
0,long_term_forecast_electricity_96_96_LightTS_c...,0.2133,0.3156,0.0013
1,long_term_forecast_electricity_96_192_LightTS_...,0.2218,0.3251,0.0018
2,long_term_forecast_electricity_96_336_LightTS_...,0.2416,0.3445,0.0023
3,long_term_forecast_electricity_96_720_LightTS_...,0.2770,0.3710,0.0037
4,long_term_forecast_electricity_192_96_LightTS_...,0.1811,0.2904,0.0019
...,...,...,...,...
110,long_term_forecast_electricity_96_720_Informer...,0.3921,0.4558,0.0043
111,long_term_forecast_electricity_192_96_Informer...,0.3274,0.4108,0.0016
112,long_term_forecast_electricity_192_192_Informe...,0.3685,0.4482,0.0023
113,long_term_forecast_electricity_192_336_Informe...,0.3720,0.4507,0.0046


In [24]:
df['Methods'] = df['Experiment_number'].str.split('_').str[6] # get name of models
method_names = df['Methods'].unique()  # get this for later use
df['Horizon'] = df['Experiment_number'].str.split('_').str[5].astype(int) # get the horizon
horizon = df['Horizon'].unique()  # get this for later use
df['Look_back'] = df['Experiment_number'].str.split('_').str[4].astype(int) # get look back window or seq length
look_back = df['Look_back'].unique()  # get this for later use
df['Experiment'] = df['Experiment_number'].str.split('_').str[-1].astype(int) # get the experiment number
df.drop('Experiment_number', axis=1, inplace=True) # drop the experiment column
df.head() # turn this into sample

,MSE,MAE,Inference_time,Methods,Horizon,Look_back,Experiment
0,0.2133,0.3156,0.0013,LightTS,96,96,0
1,0.2218,0.3251,0.0018,LightTS,192,96,0
2,0.2416,0.3445,0.0023,LightTS,336,96,0
3,0.2770,0.3710,0.0037,LightTS,720,96,0
4,0.1811,0.2904,0.0019,LightTS,96,192,0


In [25]:
df.set_index(['Methods', 'Horizon', 'Look_back', 'Experiment'], inplace=True)
df.head(n=10) # turn this into sample

MSE     MAE  Inference_time
Methods      Horizon Look_back Experiment                                
LightTS      96      96        0           0.2133  0.3156          0.0013
             192     96        0           0.2218  0.3251          0.0018
             336     96        0           0.2416  0.3445          0.0023
             720     96        0           0.2770  0.3710          0.0037
             96      192       0           0.1811  0.2904          0.0019
             192     192       0           0.2020  0.3118          0.0019
             336     192       0           0.2287  0.3375          0.0029
             720     192       0           0.2617  0.3594          0.0034
iTransformer 96      96        0           0.1488  0.2406          0.0034
             192     96        0           0.1645  0.2561          0.0027

In [26]:
df_stat = df.groupby(['Methods', 'Horizon', 'Look_back']).agg(['mean', 'max']) # getting different statistics
df_stat = df.groupby(['Methods', 'Horizon', 'Look_back']).agg(['mean']) # getting different statistics
df_stat

MSE     MAE Inference_time
                                  mean    mean           mean
Methods      Horizon Look_back                               
Autoformer   96      96         0.2023  0.3190         0.0040
                     192        0.1979  0.3134         0.0030
             192     96         0.2209  0.3302         0.0035
                     192        0.2174  0.3261         0.0033
             336     96         0.2409  0.3462         0.0051
...                                ...     ...            ...
iTransformer 192     192        0.1554  0.2494         0.0037
             336     96         0.1784  0.2711         0.0040
                     192        0.1705  0.2664         0.0043
             720     96         0.2275  0.3117         0.0050
                     192        0.1972  0.2895         0.0078

[115 rows x 3 columns]

In [27]:
# Get the best results for each pred step and look back window

# Define the slices
idx = pd.IndexSlice 
slices = [idx[idx[:, pred_len, look_b], idx[:, :]] for look_b in look_back for pred_len in horizon]

# Apply highlighting for each slice using a loop
styled_df = df_stat.style
for sclice in slices:
    styled_df = styled_df.highlight_min(axis=0, props='background-color:green; font-weight:bold;', subset=sclice)

# Format and display
styled_df = styled_df.format(precision=3)
styled_df # how to get only the highlighted part or rows around it?

In [ ]:



def highlight_best(selected_look_back, selected_metric):

    idx = pd.IndexSlice 
    selected_stats = df_stat.loc[idx[:,:,selected_look_back],idx[:,selected_metric]] 
    # drop the look_back window since it is the same for everyone
    selected_stats = selected_stats.droplevel(2, axis=0) 
    # drop the mean name because it is the same for all the columns
    selected_stats = selected_stats.droplevel(1, axis=1) 
    # reshape to the wide format
    selected_stats = selected_stats.unstack(level=0)
    # swap the names of the columns
    selected_stats = selected_stats.swaplevel(0, 1, axis=1)
    # set the column names and sort them
    selected_stats.columns.names = ['Methods', 'Metrics']
    selected_stats.sort_index(axis=1, inplace=True)


    # shuffling models for the final print, change the order in method names for 
    # putting similar methods beside each others!
    metric_names = ['MSE', 'MAE']
    idx = pd.IndexSlice 
    selected_stats = selected_stats.loc[:,idx[method_names,metric_names]]

    # get the best models and highlight them
    idx = pd.IndexSlice 
    sclice_1 = idx[idx[:], idx[:,'MSE']]
    sclice_2 = idx[idx[:], idx[:,'MAE']]


    styled_df = selected_stats.style.highlight_min(axis=1,
                                        props='background-color:green; font-weight:bold;', subset=sclice_1)\
                        .highlight_min(axis=1,props='background-color:green; font-weight:bold;', subset=sclice_2 )\
                        .format(precision=3)
    
        # get a latex friendly output;
    num_col = len(selected_stats.iloc[0]) + 1
    
    s = styled_df.to_latex(convert_css=True, hrules=True, 
                            column_format=f"{num_col*'c'}",
                            multicol_align='c'
                            )




    for (header, text_color) in METHODS_COLORS.items():
        new_header = r"\color{" + text_color +"}"  + header
        s = s.replace(header, new_header)


    f = open("table.txt", 'a')
    f.write(s+ "\n" + 'new table' + '\n')
    f.close()
    return styled_df, selected_stats
# we need to take a screenshot for uploading to github



In [29]:
# only selecting a specific look back and mean of data for reporting

# color the name of similar methods
COLORS = {'transformer':'RoyalBlue', 
          'CNN':'BrickRed',
         'MLP':'OliveGreen',
         'RNN':'SkyBlue',
         'statespace':'Orchid'}

METHODS_COLORS = {'DLinear':COLORS['MLP'],
                 'TSMixer':COLORS['MLP'],
                 'TimeMixer':COLORS['MLP'],
                 'Koopa':COLORS['MLP'],
                 'N-BEATS':COLORS['MLP'],
                 'Autoformer':COLORS['transformer'],
                 'Informer':COLORS['transformer'],
                 'PatchTST':COLORS['transformer'],
                 'Crossformer':COLORS['transformer'],
                 'Reformer':COLORS['transformer'],
                 'TFT':COLORS['transformer'],
                 'FEDformer':COLORS['transformer'],
                 'Pyraformer':COLORS['transformer'],
                 'Mamba':COLORS['statespace'],
                 'MICN':COLORS['CNN'],
                 'TimesNet':COLORS['CNN'],
                 'SegRNN':COLORS['RNN'],
                 'ETSformer':COLORS['transformer'],
                 'iTransformer':COLORS['transformer'],
                 'LightTS':COLORS['MLP']
                 }


In [30]:

styled_df, selected_stats = highlight_best(look_back[0],'mean' )
styled_df

In [31]:
styled_df, selected_stats = highlight_best(look_back[1],'mean' )
styled_df

/tmp/ipykernel_2217070/276775383.py:22: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  selected_stats = selected_stats.loc[:,idx[method_names,metric_names]]
